In [1]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from transformers import BartTokenizer, BartForConditionalGeneration

# Carga del dataset
df_ref = pd.read_csv("df_ref.csv")

model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

bleu_scores = []

def generate_summary_bart(article, reference_summary):
    input_text = article
    inputs = tokenizer([input_text], return_tensors='pt', max_length=1024, truncation=True)
    
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    reference_tokens = tokenizer.tokenize(reference_summary)
    hypothesis_tokens = tokenizer.tokenize(generated_summary)
    bleu = sentence_bleu([reference_tokens], hypothesis_tokens)
    
    return generated_summary, bleu

for index, row in df_ref.iterrows():
    article = row['article']
    reference_summary = row['abstract']
    
    generated_summary, bleu = generate_summary_bart(article, reference_summary)
    
    bleu_scores.append(bleu)

bleu_scores_df = pd.DataFrame({
    'BLEU': bleu_scores
})

pd.options.display.float_format = '{:.10f}'.format

print(bleu_scores_df)


C:\Users\oscarandres.pinilla\AppData\Local\Temp\ipykernel_5176\372804217.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\oscarandres.pinilla\anaconda3\envs\tfm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\oscarandres.pinilla\anaconda3\envs\tfm\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, i

           BLEU
0  0.0029901203
1  0.0000000000
2  0.0586954039
3  0.0468462092
4  0.0029341966
5  0.0407869602
6  0.0743142740
7  0.0043172680
8  0.0006167773
9  0.0410075689
10 0.0863882720
11 0.0366385686
12 0.0087297565
13 0.0391200870
14 0.1104265840
15 0.0124022966
16 0.0271067729
17 0.0620407736
18 0.2142814357
19 0.1002024347


In [2]:
bleu_scores_df

,BLEU
0,0.0029901203
1,0.0000000000
2,0.0586954039
3,0.0468462092
4,0.0029341966
5,0.0407869602
6,0.0743142740
7,0.0043172680
8,0.0006167773
9,0.0410075689


In [3]:
# Para posteriores cálculos se extrae el dataframe 
bleu_scores_df.to_csv('BLEU_scores_BART_df.csv', index=False)